<a href="https://colab.research.google.com/github/RochanaChaturvedi/Politext/blob/main/Ideal_point_universal_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

In [ ]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics import confusion_matrix


from absl import logging
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset

In [ ]:
df=pd.read_csv("/content/drive/My Drive/IdealPointEstimation/80_114_bills.csv")
df.title.replace('(\"\")+',"\"",regex=True,inplace=True)
df.title=df.title.apply(lambda text: text.strip())
df.title=df.title.apply(lambda text:re.sub("\s{2,}"," ",text))

In [ ]:
# #some exploration
# print(df.private.value_counts())
# print(df.billtype.value_counts())
# df.party.value_counts()

#remove bills containg text "the relief of" as they won't reflect ideology
df.billtype=df.billtype.apply(str.lower)
df=df[~ df.title.str.contains("or the relie")] 
df=df[df.pooleid.notnull()]
df=df[df.pooleid!=0]
df=df[df.dw1.notnull()]
df.reset_index(inplace=True,drop=True)#imp
df.set_index("index",inplace=True)
df.head()

,billid,billtype,chamber,cong,plaw,private,title,dw1,memberid,party,pooleid,passh,passs,majority,major
index,,,,,,,,,,,,,,,
0,100-HCONRES-1,hconres,0,100,0,0.0,A concurrent resolution providing for a Joint ...,-0.396,10726-100-100,100.0,10726.0,1,1,1.0,NaN
1,100-HCONRES-10,hconres,0,100,0,0.0,A concurrent resolution expressing the sense o...,0.460,14414-100-200,200.0,14414.0,0,0,0.0,NaN
2,100-HCONRES-100,hconres,0,100,0,0.0,A concurrent resolution recognizing and celebr...,-0.533,13035-100-100,100.0,13035.0,0,0,1.0,NaN
3,100-HCONRES-101,hconres,0,100,0,0.0,A concurrent resolution expressing the sense o...,0.078,14863-100-200,200.0,14863.0,1,0,0.0,NaN
4,100-HCONRES-102,hconres,0,100,0,0.0,A concurrent resolution setting forth the cong...,0.257,14668-100-200,200.0,14668.0,0,0,0.0,NaN


### SEE CHARACTERS 
for preprocessing decisions

In [ ]:
from collections import Counter
from keras.preprocessing.text import Tokenizer


def tokenize(x):
  unique_symbols = Counter()

  for _, name in x.iteritems():
      unique_symbols.update(name)

  num_unique_symbols = len(unique_symbols) #- len(uncommon_symbols) + 1 
  print("Unique symbols:",num_unique_symbols, unique_symbols)

  tokenizer = Tokenizer(
      char_level=True,
      filters=None,
      oov_token='#',
      lower=False,
      num_words=None
  )

  tokenizer.fit_on_texts([x])
  char_dict = {}
  ind_to_char={}
  for i, char in enumerate(unique_symbols):
    char_dict[char] = i + 1
    ind_to_char[i+1]=char
  tokenizer.word_index = char_dict.copy()
  print(char_dict)
tokenize(df.title)

Unique symbols: 27 Counter({' ': 11588863, 'e': 7596685, 't': 6171385, 'o': 5583039, 'a': 5039525, 'i': 4871483, 'r': 4580953, 'n': 4566887, 's': 3627767, 'l': 2469107, 'd': 2393007, 'c': 2279555, 'h': 2095377, 'f': 1991421, 'u': 1608724, 'p': 1591082, 'm': 1572066, 'g': 917909, 'b': 806999, 'v': 731017, 'y': 682712, 'w': 355981, 'x': 172761, 'k': 153965, 'z': 113667, 'j': 95260, 'q': 74766})
{'a': 1, ' ': 2, 'c': 3, 'o': 4, 'n': 5, 'u': 6, 'r': 7, 'e': 8, 't': 9, 's': 10, 'l': 11, 'i': 12, 'p': 13, 'v': 14, 'd': 15, 'g': 16, 'f': 17, 'j': 18, 'm': 19, 'h': 20, 'x': 21, 'w': 22, 'b': 23, 'y': 24, 'q': 25, 'z': 26, 'k': 27}


# Universal Sentence encoder : mean of sentence embeddings

In [ ]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#"https://tfhub.dev/google/universal-sentence-encoder-large/5") #preprocessing not required
def embed(input):
  return model(input)
logging.set_verbosity(logging.ERROR)

In [ ]:
df["embeddings"] = np.nan
nrows = len(df)
batch_size = 4000 #process in batches to keep in sync with available RAM
df.to_csv('/content/drive/My Drive/IdealPointEstimation/80_114_univ_sent_emb.csv',index=True)
nrows

In [ ]:
for i in range(0,nrows//batch_size):
  data_batch=df.loc[(df.index >= i*batch_size) & (df.index < (i+1)*batch_size)]
  data_batch["embeddings"]=embed(tf.constant(data_batch.title.to_list()))
  data_batch.embeddings=data_batch.embeddings.apply(lambda x:np.array(x))
  # df.embeddings.iloc[i*batch_size:(i+1)*batch_size]=data_batch.embeddings
  df=pd.read_csv('/content/drive/My Drive/IdealPointEstimation/80_114_univ_sent_emb.csv',index_col="index") 
  df.loc[(df.index >= i*batch_size) & (df.index < (i+1)*batch_size),'embeddings']=data_batch.embeddings
  df.to_csv('/content/drive/My Drive/IdealPointEstimation/80_114_univ_sent_emb.csv',index=True)
  df=df[df.embeddings.isnull()]
  print("Processed"+str(i+1)+"batches")

In [ ]:
#last batch of size <batch_size
data_batch=df.loc[(df.index >= nrows//batch_size*batch_size) & (df.index < nrows)]
data_batch["embeddings"]=embed(tf.constant(data_batch.title.to_list()))
data_batch.embeddings=data_batch.embeddings.apply(lambda x:np.array(x))
print("Processed last batch")
df=pd.read_csv('/content/drive/My Drive/IdealPointEstimation/80_114_univ_sent_emb.csv',index_col="index") 
df.loc[(df.index >= nrows//batch_size*batch_size) & (df.index < nrows),'embeddings']=data_batch.embeddings
df.to_csv('/content/drive/My Drive/IdealPointEstimation/80_114_univ_sent_emb.csv',index=True)

#remaining rows
df=df[df.embeddings.isnull()]                                                   
df

Processed last batch


,billid,billtype,chamber,cong,plaw,private,title,dw1,memberid,party,pooleid,passh,passs,majority,major,embeddings
index,,,,,,,,,,,,,,,,


# Load and clean embedding

In [ ]:
# df=pd.read_csv("/content/drive/My Drive/IdealPointEstimation/80_114_univ_sent_emb.csv",index_col="index")

In [ ]:
def clean_embedding(x):
  x=x.replace("\n","")
  x=x.replace("[","")
  x=x.replace("]","")
  x=np.array(list(map(float, x.split())))
  return x
df.embeddings=df.embeddings.apply(clean_embedding)
df=df[df.embeddings.notnull()]
df.to_csv('/content/drive/My Drive/IdealPointEstimation/80_114_univ_sent_emb.csv',index=True)
# len(df)

# Projections

## plot embeddings

In [ ]:
def plot_embedding(tokens,label):
  plt.figure(figsize=(15, 10)) 
  x=tokens[:,0]
  y=tokens[:,1]
  #iterate over symbols
  for i in range(len(x)):
        plt.scatter(x[i],y[i],c=['r','g'])
        plt.text(x[i], y[i], label[i+1],verticalalignment='bottom', horizontalalignment='right',fontsize=15)
        plt.legend(label)
  plt.show()

In [ ]:
# Standardizing the features: scaling the embeddings
df['all']='A'
mean=df.groupby('all')['embeddings'].apply(np.mean)[0]
df['std']=df['embeddings'].apply(lambda x: abs(x-mean)**2)
std=np.sqrt(df.groupby('all')['std'].apply(np.sum)[0])/(len(df)-1)
df['scaled']=df.embeddings.apply(lambda x:np.divide((x-mean),std))

In [ ]:
pca = PCA(n_components=100)

q=np.concatenate(df.scaled.values)
ncol=512
# nrow=len(df)
q=q.reshape((-1,ncol))
q = pca.fit_transform(q)

tsne_results= TSNE(n_components=2,n_jobs=-1,perplexity=50).fit_transform(q)
principalDf = pd.DataFrame(data = q, columns = ['comp1', 'comp2'])
principalDf = pd.concat([principalDf, df[['party']]], axis = 1)

fig = plt.figure(figsize = (15,10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('PCA', fontsize = 20)
targets = ['Democrat', 'Republican']
target_code = [100.0,200.0]
colors = ['r', 'b']
for target, color in zip(target_code,colors):
    indicesToKeep = principalDf['party'] == target
    ax.scatter(principalDf.loc[indicesToKeep, 'comp1']
               , principalDf.loc[indicesToKeep, 'comp2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()
plt.show()

#Using TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(weights)

tsne = TSNE(n_components=2, random_state=21154,perplexity=50,init='pca', n_iter=100000)#, method='exact')
tsne_results= tsne.fit_transform(embeddings)
tsne_results= tsne.fit_transform(pca_result)

plot_embedding(tsne_results,party)

## Some Utility Functions

In [ ]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a bill vector of shape (n,)          
        v -- a bill vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u,v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.linalg.norm(u)
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.linalg.norm(v)
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot/(norm_u*norm_v)
    
    return cosine_similarity


#test
obamacare=df.embeddings[df.billid=="111-HR-3590"].iloc[0]

repeal = df.embeddings[df.billid=="114-HR-3762"].iloc[0]
obamacare2 =df.embeddings[df.billid=="114-HR-4872"].iloc[0]
repeal2 = df.embeddings[df.billid=="112-HR-2"].iloc[0]
print(obamacare.shape)

print("cosine_similarity(obamacare, obamacare2) = ", cosine_similarity(obamacare, obamacare2))
print("cosine_similarity(obamacare, repeal) = ",cosine_similarity(obamacare, repeal))
print("cosine_similarity(obamacare, repeal2) = ",cosine_similarity(obamacare, repeal2))

cosine_similarity(obamacare, obamacare2) =  0.14446945877949102
cosine_similarity(obamacare, repeal) =  0.06726060460514993
cosine_similarity(obamacare, repeal2) =  0.41620337493714643


In [ ]:
def analogy(bill_a, bill_b, bill_c, bill_to_vec_map):
    """
    Performs the vec analogy task: a is to b as c is to ____. 
    
    Arguments:
    bill_a -- a billid, 
    bill_b -- a billid, 
    bill_c -- a billid,
    bill_to_vec_map -- dataframe that maps bills to their corresponding vectors. 
    
    Returns:
    best_bill --  the vec such that v_b - v_a is close to v_best_vec - v_c, as measured by cosine similarity
    """
    
    convert bills to lower case
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    # Get the bill embeddings v_a, v_b and v_c (≈1-3 lines)
    e_a, e_b, e_c = bill_to_vec_map['billId=bill_a],bill_to_vec_map[bill_b],bill_to_vec_map[bill_c]
    
    bills = bill_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_bill = None                   # Initialize best_bill with None, it will help keep track of the bill to output

    # loop over the whole bill vector set
    for w in bills:        
        # to avoid best_bill being one of the input bills, pass on them.
        if w in [bill_a, bill_b, bill_c] :
            continue
        
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = cosine_similarity(e_b-e_a,bill_to_vec_map[w]-e_c)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_bill to the current bill (≈3 lines)
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_bill = w        
    return best_bill

## Calculate Bias Axis

In [ ]:
X=df.groupby('party')['embeddings'].apply(np.mean)
g = X.iloc[1] - X.iloc[0]
g

array([-2.06099314e-02,  1.96841178e-04,  3.02108102e-03, -8.03356359e-03,
        6.82764286e-03,  3.56553239e-03,  3.84433622e-03,  1.20044605e-02,
       -1.79739093e-02, -2.46447451e-02,  8.16902332e-03, -1.55382524e-02,
        1.18113839e-02,  5.84858267e-03,  1.58901915e-03, -1.92253325e-04,
        1.53015528e-02,  1.18012054e-02,  1.63264406e-02,  2.83786589e-03,
       -2.13854843e-02,  7.54488889e-03,  1.22028773e-02, -1.26343189e-02,
        8.46585104e-03, -2.03569904e-03, -1.71634811e-03, -9.39324483e-03,
       -2.66262187e-03, -1.24977694e-02, -8.77996933e-03, -9.73932726e-03,
       -2.70051707e-02, -2.99338895e-03,  1.81787240e-02,  9.48256675e-03,
        2.14586496e-02,  1.56661145e-02, -1.58477818e-02,  1.08629888e-02,
        5.43472935e-03, -3.65443277e-03, -8.23926424e-03, -7.07715486e-03,
        1.06168498e-02, -1.75883065e-02,  1.13576490e-02,  8.84804259e-03,
        2.32582638e-02, -6.64706283e-03, -1.85541915e-02, -1.49400255e-02,
       -2.42709522e-03, -

## Projection

If you're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction  g , and the remaining 49 dimensions, which we'll call  g⊥ . In linear algebra, we say that the 49 dimensional  g⊥  is perpendicular (or "othogonal") to  g , meaning it is at 90 degrees to  g . The neutralization step takes a vector such as  ereceptionist  and zeros out the component in the direction of  g , giving us  edebiasedreceptionist .
Given an input embedding $e$, you can use the following formulas to compute $e^{debiased}$: 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$


In [ ]:
def project(bill, g):
    """
    Removes the bias of "bill" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that ideology neutral bills are zero in the ideology subspace.
    
    Arguments:
        bill -- billid indicating the bill to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as ideology)
    
    Returns:
        e_debiased -- neutralized bill vector representation of the input "bill"
    """
    
    # Select bill vector representation of "bill". Use bill_to_vec_map. (≈ 1 line)
    e = bill
    
    # Compute e_biascomponent using the formula give above. (≈ 1 line). This is jnothing but projection of e on g
    e_biascomponent = np.dot(g,e)/(np.linalg.norm(g))
 
    # Neutralize e by subtracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection. (≈ 1 line)
    # e_debiased = e-e_biascomponent
    
    return e_biascomponent

In [ ]:
df["bias_comp"]=df.embeddings.apply(lambda x:project(x,g))
len(df)

437069

In [ ]:
#calculate mean projections of a member
df['mem_mean_biascomp']=df.groupby(['memberid','chamber'])['bias_comp'].transform(np.mean)#,num_bills=('count']})
df['num_bills']=df.groupby(['memberid','chamber'])['billid'].transform('count')
df.drop_duplicates(['memberid','chamber'],inplace=True)

In [ ]:
from scipy.stats import spearmanr
print("pearson's rho: ",str(df[df.num_bills>0].dw1.corr(df[df.num_bills>0].mem_mean_biascomp, method="pearson")))
coef, p = spearmanr(df[df.num_bills>0].dw1, df[df.num_bills>0].mem_mean_biascomp)
print("spearman coef: ",coef,"pvalue: ",p)

pearson's rho:  0.02490654998485802
spearman coef:  0.014864595271850571 pvalue:  0.04318228606423421


In [ ]:
print(confusion_matrix(df.dw1>0,df.mem_mean_biascomp>0))

[[ 520 9069]
 [ 464 8450]]


In [ ]:
for billtype in df.billtype.unique():# major.unique():
  dftype=df[df.billtype==billtype]
  coef, p = spearmanr(dftype.dw1, dftype.mem_mean_biascomp)
  print(billtype, "#bills: ",len(dftype),"spearman coef: ",coef,"pvalue: ",p)

hconres #bills:  4128 spearman coef:  -0.014565009392302392 pvalue:  0.34949927109771173
hjres #bills:  1814 spearman coef:  0.02681118242836146 pvalue:  0.2537304510541021
hr #bills:  9151 spearman coef:  0.04180011755955475 pvalue:  6.33861228868214e-05
hres #bills:  16 spearman coef:  0.2764705882352941 pvalue:  0.2999421284923735
s #bills:  3392 spearman coef:  0.1320566084137129 pvalue:  1.1443016314687173e-14
               billid billtype  ...  mem_mean_biascomp  num_bills
index                           ...                              
43282  103-SCONRES-29  sconres  ...           0.109696          1

[1 rows x 19 columns]
sconres #bills:  1 spearman coef:  nan pvalue:  nan
sres #bills:  1 spearman coef:  nan pvalue:  nan


In [ ]:
coef, p = spearmanr(df[abs(df.mem_mean_biascomp)>0.1].dw1, df[abs(df.mem_mean_biascomp)>0.1].mem_mean_biascomp)
print("spearman coef: ",coef,"pvalue: ",p)  

spearman coef:  0.03313495827889663 pvalue:  0.002792711182867651


In [ ]:
df.mem_mean_biascomp.describe()

count    13040.000000
mean         0.014860
std          0.058894
min         -0.306417
25%         -0.017220
50%          0.018173
75%          0.052110
max          0.278722
Name: mem_mean_biascomp, dtype: float64

In [ ]:
df[df.bias_comp>0.4]

,billid,billtype,chamber,cong,plaw,private,title,dw1,memberid,party,pooleid,passh,passs,majority,major,embeddings,bias_comp,mem_mean_biascomp,num_bills
index,,,,,,,,,,,,,,,,,,,
2894,100-HR-2719,hr,0,100,0,0.0,"A bill to amend title 10, United States Code, ...",0.360,10717-100-200,200.0,10717.0,0,0,0.0,2.0,"[-0.04744902, -0.0698745, 0.0188491, -0.022763...",0.457698,0.327545,2
7077,100-S-1066,s,1,100,0,0.0,A bill to extend the suspension of duties on c...,0.161,14501-100-200,200.0,14501.0,0,0,0.0,18.0,"[-0.0156174898, 0.0326210037, 0.0551719591, -0...",0.428839,0.173920,37
7079,100-S-1068,s,1,100,0,0.0,A bill to amend the Clayton Act regarding inte...,-0.511,14073-100-100,100.0,14073.0,0,1,1.0,15.0,"[0.06265023, 0.01657869, 0.00919163, -0.051895...",0.402259,0.190668,58
14740,101-HR-3595,hr,0,101,0,0.0,To protect the retired pay of certain members ...,0.359,10717-101-200,200.0,10717.0,0,0,0.0,16.0,"[-0.01385368, 0.0128472, 0.0216365, -0.0533825...",0.435816,0.149128,5
18321,101-S-102,s,1,101,0,0.0,A bill to authorize reduced postage rates for ...,-0.339,4812-101-100,100.0,4812.0,0,0,1.0,20.0,"[-0.03237223, -0.03722857, 0.05400043, -0.0765...",0.422125,0.205402,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412802,97-HR-2605,hr,0,97,0,0.0,A bill to amend the Internal Revenue Code of 1...,-0.430,14649-97-100,100.0,14649.0,0,0,1.0,20.0,"[0.0139003759, -0.0313721299, -0.0315611139, -...",0.402019,0.203807,19
418939,97-S-1235,s,1,97,0,0.0,A bill to exempt certain matters relating to t...,0.119,14900-97-200,200.0,14900.0,0,0,1.0,16.0,"[0.00960022304, 0.000786330143, 0.0231300946, ...",0.429471,0.148403,42
419028,97-S-1324,s,1,97,0,0.0,"A bill to amend section 1751 of title 18, Unit...",-0.288,1366-97-100,100.0,1366.0,0,0,0.0,12.0,"[0.00202536606, -0.0554484241, 0.00160731818, ...",0.411330,0.145111,49


In [ ]:
df[df.bias_comp<-0.4]

,index,billid,billtype,chamber,cong,plaw,private,title,dw1,memberid,party,pooleid,passh,passs,majority,major,embeddings,bias_comp
9646,9646,100-S-872,s,1,100,0,0.0,A bill to temporarily suspend the duty on 4-ch...,0.009,14500-100-200,200.0,14500.0,0,0,0.0,18.0,"[0.0156317167, 0.0709056631, 0.0231965091, -0....",-0.402820
19792,19792,101-S-2376,s,1,101,0,0.0,A bill to temporarily suspend the duty on 4-ch...,0.013,14500-101-200,200.0,14500.0,0,0,0.0,18.0,"[0.0156317204, 0.070905678, 0.0231965221, -0.0...",-0.402820
30708,30708,102-S-1635,s,1,102,0,0.0,A bill to suspend temporarily the duty on 4-(6...,-0.417,14914-102-100,100.0,14914.0,0,0,1.0,18.0,"[-0.00530436, 0.07750946, -0.04014782, -0.0211...",-0.400653
32310,32310,102-S-3103,s,1,102,0,0.0,"A bill to suspend until January 1, 1995, the d...",0.389,14806-102-200,200.0,14806.0,0,0,0.0,18.0,"[-0.0174600482, 0.0469147339, -0.0741527826, 0...",-0.405630
40988,40988,103-S-1220,s,1,103,0,0.0,A bill to suspend temporarily the duty on exom...,0.403,14806-103-200,200.0,14806.0,0,0,0.0,18.0,"[0.0080777, 0.06140022, -0.04672975, -0.013319...",-0.419695
49786,49786,104-S-161,s,1,104,0,0.0,A bill to amend the Internal Revenue Code of 1...,-0.421,49308-104-100,100.0,49308.0,0,0,0.0,15.0,"[0.06134765, -0.050989, -0.06391357, -0.051928...",-0.400666
58356,58356,105-S-1661,s,1,105,0,0.0,A bill to suspend temporarily the duty on N-Et...,-0.367,14508-105-100,100.0,14508.0,0,0,0.0,18.0,"[0.024324026, 0.0231633503, 0.0220597573, 0.01...",-0.412309
62711,62711,106-HR-2255,hr,0,106,0,0.0,To amend the Internal Revenue Code of 1986 to ...,-0.407,29571-106-100,100.0,29571.0,0,0,0.0,1.0,"[0.02482025, -0.03984565, -0.06475835, -0.0536...",-0.419392
69543,69543,106-S-2808,s,1,106,0,0.0,A bill to amend the Internal Revenue Code of 1...,0.298,49500-106-200,200.0,49500.0,0,0,1.0,8.0,"[0.04453562, 0.01933435, -0.0653671, -0.036637...",-0.428195
83792,83792,108-HR-1968,hr,0,108,0,0.0,To suspend temporarily the duty on Butyl Ethyl...,-0.371,15029-108-100,100.0,15029.0,0,0,0.0,18.0,"[0.03939994, 0.07066688, 0.02648431, 0.0008221...",-0.403094
